In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [3]:
events_df_train = pd.read_csv('data/event_data_train.csv')

In [4]:
events_df_train['date'] = pd.to_datetime(events_df_train.timestamp, unit='s')
events_df_train['day'] = events_df_train.date.dt.date

In [5]:
events_df_train.head()

,step_id,timestamp,action,user_id,date,day
0,32815,1434340848,viewed,17632,2015-06-15 04:00:48,2015-06-15
1,32815,1434340848,passed,17632,2015-06-15 04:00:48,2015-06-15
2,32815,1434340848,discovered,17632,2015-06-15 04:00:48,2015-06-15
3,32811,1434340895,discovered,17632,2015-06-15 04:01:35,2015-06-15
4,32811,1434340895,viewed,17632,2015-06-15 04:01:35,2015-06-15


In [6]:
users_data = events_df_train.pivot_table(index='user_id',
                                         columns='action',
                                         values='step_id', 
                                         aggfunc='count',
                                         fill_value=0).reset_index()

In [7]:
users_data.head()

action,user_id,discovered,passed,started_attempt,viewed
0,1,1,0,0,1
1,2,9,9,2,10
2,3,91,87,30,192
3,5,11,11,4,12
4,7,1,1,0,1


In [8]:
is_gone_count = users_data.loc[users_data.passed >= 40].user_id.count()
all_users_count = users_data.user_id.count()
is_gone_count/all_users_count

0.24706249350109183

In [9]:
users_data['success'] = users_data.passed >= 40
users_data['success'] = users_data['success'].apply(int)

In [10]:
users_data.head()

action,user_id,discovered,passed,started_attempt,viewed,success
0,1,1,0,0,1,0
1,2,9,9,2,10,0
2,3,91,87,30,192,1
3,5,11,11,4,12,0
4,7,1,1,0,1,0


In [11]:
unique_days = events_df_train.groupby('user_id', as_index=False).agg({'day':'nunique'}).rename(columns={'day':'unique_days'})

In [12]:
users_data = users_data.merge(unique_days, on='user_id', how='outer')

In [13]:
users_data.head()

,user_id,discovered,passed,started_attempt,viewed,success,unique_days
0,1,1,0,0,1,0,1
1,2,9,9,2,10,0,2
2,3,91,87,30,192,1,7
3,5,11,11,4,12,0,2
4,7,1,1,0,1,0,1


In [14]:
users_data.isnull().sum()

user_id            0
discovered         0
passed             0
started_attempt    0
viewed             0
success            0
unique_days        0
dtype: int64

In [15]:
sub_data_train = pd.read_csv('data/submissions_data_train.csv')

In [16]:
sub_data_train.head()

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


In [17]:
sub_data_train['date'] = pd.to_datetime(sub_data_train.timestamp, unit='s')
sub_data_train['day'] = sub_data_train.date.dt.date

In [18]:
sub_data_train.head()

,step_id,timestamp,submission_status,user_id,date,day
0,31971,1434349275,correct,15853,2015-06-15 06:21:15,2015-06-15
1,31972,1434348300,correct,15853,2015-06-15 06:05:00,2015-06-15
2,31972,1478852149,wrong,15853,2016-11-11 08:15:49,2016-11-11
3,31972,1478852164,correct,15853,2016-11-11 08:16:04,2016-11-11
4,31976,1434348123,wrong,15853,2015-06-15 06:02:03,2015-06-15


In [19]:
users_sub_data = sub_data_train.pivot_table(index='user_id',
                                            columns='submission_status',
                                            values='step_id',
                                            aggfunc='count',
                                            fill_value=0).reset_index()

In [20]:
users_sub_data.head()

submission_status,user_id,correct,wrong
0,2,2,0
1,3,29,23
2,5,2,2
3,8,9,21
4,14,0,1


In [21]:
users_sub_data['accuracy'] = users_sub_data.correct / (users_sub_data.correct + users_sub_data.wrong)

In [22]:
users_sub_data.head()

submission_status,user_id,correct,wrong,accuracy
0,2,2,0,1.000000
1,3,29,23,0.557692
2,5,2,2,0.500000
3,8,9,21,0.300000
4,14,0,1,0.000000


In [23]:
users_data = users_data.merge(users_sub_data, on='user_id', how='outer').fillna(0)

In [24]:
users_data.head()

,user_id,discovered,passed,started_attempt,viewed,success,unique_days,correct,wrong,accuracy
0,1,1,0,0,1,0,1,0.0,0.0,0.000000
1,2,9,9,2,10,0,2,2.0,0.0,1.000000
2,3,91,87,30,192,1,7,29.0,23.0,0.557692
3,5,11,11,4,12,0,2,2.0,2.0,0.500000
4,7,1,1,0,1,0,1,0.0,0.0,0.000000


In [25]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [26]:
X = users_data.drop(columns=['success', 'user_id'])
y = users_data.success

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 50)

In [28]:
rf_clf = RandomForestClassifier(criterion='entropy')

In [29]:
params = {'n_estimators':range(30,100,10), 'max_depth':range(2,12,2), 'min_samples_split':range(2,30,5), 'min_samples_leaf':range(2,30,2)}

In [30]:
grid_scv = RandomizedSearchCV(rf_clf, params, cv=5)

In [31]:
grid_scv.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(criterion='entropy'),
                   param_distributions={'max_depth': range(2, 12, 2),
                                        'min_samples_leaf': range(2, 30, 2),
                                        'min_samples_split': range(2, 30, 5),
                                        'n_estimators': range(30, 100, 10)})

In [32]:
best_clf = grid_scv.best_estimator_

In [33]:
prediction = best_clf.predict(X_test)

In [34]:
from sklearn.metrics import precision_score, recall_score

In [35]:
precision_score(y_test, prediction)

1.0

In [36]:
recall_score(y_test, prediction)

1.0

То есть по выбранным фичам возможно определить закончит ли студент курс или нет, теперь попробуем ограничить данные активности пользователей и возьмем только первые 2 дня

In [37]:
two_days_step = 2*24*60*60

In [38]:
first_users_timestamp = events_df_train.groupby('user_id', as_index=False).agg({'timestamp':'min'})

In [39]:
first_users_timestamp = first_users_timestamp.rename(columns={'timestamp':'first_timestamp'})

In [40]:
first_users_timestamp.head()

,user_id,first_timestamp
0,1,1472827464
1,2,1514383364
2,3,1434358476
3,5,1466156809
4,7,1521634660


In [41]:
events_df_train = events_df_train.merge(first_users_timestamp, on='user_id', how='outer')

In [42]:
events_df_train['first_two_days'] = (events_df_train.timestamp - events_df_train.first_timestamp) <= two_days_step

In [43]:
events_df_train_for_first_two_days = events_df_train.loc[events_df_train.first_two_days]

In [44]:
events_df_train_for_first_two_days.first_two_days.sum()

906203

In [45]:
events_df_train_for_first_two_days.user_id.count()

906203

In [46]:
sub_data_train = sub_data_train.merge(first_users_timestamp, on='user_id', how='outer')

In [47]:
sub_data_train['first_two_days'] = (sub_data_train.timestamp - sub_data_train.first_timestamp) <= two_days_step

In [48]:
sub_data_train.head()

,step_id,timestamp,submission_status,user_id,date,day,first_timestamp,first_two_days
0,31971.0,1.434349e+09,correct,15853,2015-06-15 06:21:15,2015-06-15,1434345977,True
1,31972.0,1.434348e+09,correct,15853,2015-06-15 06:05:00,2015-06-15,1434345977,True
2,31972.0,1.478852e+09,wrong,15853,2016-11-11 08:15:49,2016-11-11,1434345977,False
3,31972.0,1.478852e+09,correct,15853,2016-11-11 08:16:04,2016-11-11,1434345977,False
4,31976.0,1.434348e+09,wrong,15853,2015-06-15 06:02:03,2015-06-15,1434345977,True


In [49]:
sub_data_train_for_two_days = sub_data_train.loc[sub_data_train.first_two_days]

In [50]:
sub_data_train_for_two_days.head()

,step_id,timestamp,submission_status,user_id,date,day,first_timestamp,first_two_days
0,31971.0,1.434349e+09,correct,15853,2015-06-15 06:21:15,2015-06-15,1434345977,True
1,31972.0,1.434348e+09,correct,15853,2015-06-15 06:05:00,2015-06-15,1434345977,True
4,31976.0,1.434348e+09,wrong,15853,2015-06-15 06:02:03,2015-06-15,1434345977,True
5,31976.0,1.434348e+09,correct,15853,2015-06-15 06:03:08,2015-06-15,1434345977,True
7,31977.0,1.434347e+09,correct,15853,2015-06-15 05:49:31,2015-06-15,1434345977,True


In [51]:
users_data_for_two_days = events_df_train_for_first_two_days.pivot_table(index='user_id',
                                         columns='action',
                                         values='step_id', 
                                         aggfunc='count',
                                         fill_value=0).reset_index()

In [52]:
users_data_for_two_days.user_id.count()

19234

In [53]:
users_data_for_two_days = users_data_for_two_days.merge(users_data[['user_id', 'success']], on='user_id', how='outer')
unique_days = events_df_train_for_first_two_days.groupby('user_id', as_index=False).agg({'day':'nunique'}).rename(columns={'day':'unique_days'})
users_data_for_two_days = users_data_for_two_days.merge(unique_days, on='user_id', how='outer')

In [54]:
users_sub_data_ftd = sub_data_train_for_two_days.pivot_table(index='user_id',
                                            columns='submission_status',
                                            values='step_id',
                                            aggfunc='count',
                                            fill_value=0).reset_index()
users_sub_data_ftd['accuracy'] = users_sub_data_ftd.correct / (users_sub_data_ftd.correct + users_sub_data_ftd.wrong)
users_data_for_two_days = users_data_for_two_days.merge(users_sub_data_ftd, on='user_id', how='outer').fillna(0)

In [55]:
users_data_for_two_days.head()

,user_id,discovered,passed,started_attempt,viewed,success,unique_days,correct,wrong,accuracy
0,1,1,0,0,1,0,1,0.0,0.0,0.0
1,2,9,9,2,9,0,1,2.0,0.0,1.0
2,3,15,15,4,20,1,1,4.0,4.0,0.5
3,5,1,1,0,1,0,1,0.0,0.0,0.0
4,7,1,1,0,1,0,1,0.0,0.0,0.0


In [56]:
X = users_data_for_two_days.drop(columns=['success', 'user_id'])
y = users_data_for_two_days.success
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 50)

In [57]:
rf_clf = RandomForestClassifier(criterion='entropy')

In [58]:
params = {'n_estimators':range(30,100,10), 'max_depth':range(2,12,2), 'min_samples_split':range(2,30,5), 'min_samples_leaf':range(2,30,2)}

In [59]:
grid_scv = RandomizedSearchCV(rf_clf, params, cv=5)

In [60]:
grid_scv.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(criterion='entropy'),
                   param_distributions={'max_depth': range(2, 12, 2),
                                        'min_samples_leaf': range(2, 30, 2),
                                        'min_samples_split': range(2, 30, 5),
                                        'n_estimators': range(30, 100, 10)})

In [61]:
best_clf = grid_scv.best_estimator_

In [62]:
X_test.head()

,discovered,passed,started_attempt,viewed,unique_days,correct,wrong,accuracy
7977,50,48,19,124,3,17.0,19.0,0.472222
3449,1,0,0,1,1,0.0,0.0,0.000000
8038,1,1,0,1,1,0.0,0.0,0.000000
11187,57,57,26,69,3,26.0,12.0,0.684211
14219,1,1,0,1,1,0.0,0.0,0.000000


In [63]:
y_predicted_prob = best_clf.predict_proba(X_test)

In [64]:
max = 0
max_val = 0
for it in range(50):
    pred = np.where(y_predicted_prob[:,1] > it/100.0, 1,0)
    prec = precision_score(y_test, pred)
    rs = recall_score(y_test, pred)
    f1 = 2 * (prec*rs) / (rs + prec)
    if f1 > max:
        max = f1
        max_val = it/100.0

In [65]:
max_val_threshold = max_val

In [66]:
max

0.6704581358609796

In [67]:
recall_score(y_test,prediction)

1.0

In [68]:
best_clf.score(X_test, y_test)

0.8482986767485823

На тестовых данных модель показала более менее хороший результат, теперь применим модель к данным из задания

In [69]:
events_data_test = pd.read_csv('data/events_data_test.csv')

In [70]:
events_data_test['date'] = pd.to_datetime(events_data_test.timestamp, unit='s')
events_data_test['day'] = events_data_test.date.dt.date

In [71]:
events_data_test.head()

,step_id,timestamp,action,user_id,date,day
0,30456,1526893787,viewed,24417,2018-05-21 09:09:47,2018-05-21
1,30456,1526893797,viewed,24417,2018-05-21 09:09:57,2018-05-21
2,30456,1526893954,viewed,24417,2018-05-21 09:12:34,2018-05-21
3,30456,1526895780,viewed,24417,2018-05-21 09:43:00,2018-05-21
4,30456,1526893787,discovered,24417,2018-05-21 09:09:47,2018-05-21


In [72]:
sub_data_test = pd.read_csv('data/submission_data_test.csv')

In [73]:
sub_data_test['date'] = pd.to_datetime(sub_data_test.timestamp, unit='s')
sub_data_test['day'] = sub_data_test.date.dt.date

In [74]:
sub_data_test.head()

,step_id,timestamp,submission_status,user_id,date,day
0,31971,1526800961,wrong,24370,2018-05-20 07:22:41,2018-05-20
1,31971,1526800976,wrong,24370,2018-05-20 07:22:56,2018-05-20
2,31971,1526800993,wrong,24370,2018-05-20 07:23:13,2018-05-20
3,31971,1526801054,correct,24370,2018-05-20 07:24:14,2018-05-20
4,31972,1526800664,wrong,24370,2018-05-20 07:17:44,2018-05-20


In [75]:
test_user_data = events_data_test.pivot_table(index='user_id',
                                         columns='action',
                                         values='step_id', 
                                         aggfunc='count',
                                         fill_value=0).reset_index()
unique_days = events_data_test.groupby('user_id', as_index=False).agg({'day':'nunique'}).rename(columns={'day':'unique_days'})
test_user_data = test_user_data.merge(unique_days, on='user_id', how='outer')

In [76]:
test_user_data.head()

,user_id,discovered,passed,started_attempt,viewed,unique_days
0,4,1,1,0,1,1
1,6,1,1,0,1,1
2,10,2,2,0,6,1
3,12,11,9,4,14,1
4,13,70,70,35,105,2


In [77]:
test_users_sub_data = sub_data_test.pivot_table(index='user_id',
                                            columns='submission_status',
                                            values='step_id',
                                            aggfunc='count',
                                            fill_value=0).reset_index()
test_users_sub_data['accuracy'] = test_users_sub_data.correct / (test_users_sub_data.correct + test_users_sub_data.wrong)
test_user_data = test_user_data.merge(test_users_sub_data, on='user_id', how='outer').fillna(0)

In [78]:
test_user_data.head()

,user_id,discovered,passed,started_attempt,viewed,unique_days,correct,wrong,accuracy
0,4,1,1,0,1,1,0.0,0.0,0.000000
1,6,1,1,0,1,1,0.0,0.0,0.000000
2,10,2,2,0,6,1,0.0,0.0,0.000000
3,12,11,9,4,14,1,1.0,0.0,1.000000
4,13,70,70,35,105,2,29.0,36.0,0.446154


In [79]:
X_test = test_user_data.drop(columns=['user_id'])

In [98]:
y_predicted_prob = best_clf.predict_proba(X_test)

In [99]:
users_id = test_user_data.user_id.values

In [100]:
y_predicted_prob = y_predicted_prob.ravel()

In [101]:
y_predicted_prob = y_predicted_prob[1::2]

In [102]:
len(y_predicted_prob)

6184

In [103]:
len(users_id)

6184

In [104]:
result = pd.DataFrame({'user_id':users_id,'is_gone':y_predicted_prob})

In [105]:
result.head()

,user_id,is_gone
0,4,0.048564
1,6,0.048564
2,10,0.097132
3,12,0.230876
4,13,1.000000


In [106]:
result.to_csv('Results.csv', index=False)